#Libraries and Data

In [ ]:
#change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

In [ ]:
#installing libraries
!pip install pyyaml==5.4.1
!pip install darts

In [ ]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel

In [ ]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col = 0, parse_dates = True)
future_df = pd.read_csv('future.csv', index_col = 0, parse_dates = True)

In [ ]:
df.head(0)

In [ ]:
#Extract regressors
X_train = df.iloc[:,1:]
X_future = future_df.iloc[:,1:]

In [ ]:
#merge both
X = pd.concat([X_train, X_future])

In [ ]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

#Prepare for LSTM

In [ ]:
#Time Series object
series = TimeSeries.from_series(df.y)
covariates = TimeSeries.from_dataframe(X)

In [ ]:
#year
year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(),
                  freq = series.freq_str,
                  periods= X.shape[0]),
                  attribute ="year",
                  one_hot = False)

#month
month_series = datetime_attribute_timeseries(year_series,
                                            attribute ="month",
                                            one_hot = True)

#weekday
weekday_series = datetime_attribute_timeseries(year_series,
                                               attribute ="weekday",
                                               one_hot = True)

In [ ]:
# preparing scalers
transformer1 = Scaler()
transformer2 = Scaler()

In [ ]:
#Scale the Y
y_transformed = transformer1.fit_transform(series)

In [ ]:
# Scale the covariates
covariates = covariates.stack(year_series)
covariates_transformed = transformer2.fit_transform(covariates)
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

#LSTM

In [ ]:
#get the best parameters
parameters  = pd.read_csv("Forecasting Product/best_params_lstm.csv",
                          index_col = 0)
parameters

In [ ]:
#getting the parameters
n_rnn_layers = int(parameters.loc["n_rnn_layers"][0])
dropout = float(parameters.loc["dropout"][0])
hidden_dim = int(parameters.loc["hidden_dim"][0])
input_chunk_length = int(parameters.loc["input_chunk_length"][0])
lr = float(parameters.loc["lr"][0])
n_epochs = int(parameters.loc["n_epochs"][0])
training_length = int(parameters.loc["training_length"][0])

In [ ]:
#model
#https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html
model = RNNModel(model = "LSTM",
                 hidden_dim = hidden_dim,
                 n_rnn_layers= n_rnn_layers,
                 dropout = dropout,
                 n_epochs = n_epochs,
                 optimizer_kwargs = {"lr": lr},
                 random_state = 1502,
                 training_length = training_length,
                 input_chunk_length= input_chunk_length,
                 pl_trainer_kwargs={"accelerator": "gpu",
                                    "devices": [0]},
                 save_checkpoints = True)

In [ ]:
#fit the model to the data
model.fit(y_transformed,
          future_covariates = covariates_transformed)

#Predictions and exporting

In [ ]:
predictions_lstm = model.predict(n = len(future_df),
                                 future_covariates = covariates_transformed)
predictions_lstm = TimeSeries.pd_series(
    transformer1.inverse_transform(predictions_lstm)).rename("lstm")
predictions_lstm

In [ ]:
#exporting
predictions_lstm.to_csv("Forecasting Product/Ensemble/predictions_lstm.csv")